In [1]:
import os

In [2]:
%pwd

'c:\\Users\\dsawa\\Downloads\\Projects\\End-To-End-Data-Science-with-MLFlow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\dsawa\\Downloads\\Projects\\End-To-End-Data-Science-with-MLFlow'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/dev618/End-To-End-Data-Science-with-MLFlow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="dev618"
os.environ["MLFLOW_TRACKING_PASSWORD"]="adae361ef167477e68d8522bb756f98d4be5daf2"

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str



In [7]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/dev618/End-To-End-Data-Science-with-MLFlow.mlflow",
           
        )

        return model_evaluation_config

In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_tracking_uri(self.config.mlflow_uri)  # fixed set_registry_uri → set_tracking_uri
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

        # Save metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

        # Log params & metrics to MLflow
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

        # Save model locally with joblib
            model_path = "artifacts/model_evaluation/model.joblib"
            joblib.dump(model, model_path)

        # Log model artifact to MLflow (works with DagsHub)
            mlflow.log_artifact(model_path)



In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e


[2025-08-24 22:43:29,701: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-24 22:43:29,705: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-24 22:43:29,709: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-24 22:43:29,711: INFO: common: created directory at: artifacts]
[2025-08-24 22:43:29,712: INFO: common: created directory at: artifacts/model_evaluation]
[2025-08-24 22:43:31,701: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
🏃 View run marvelous-bear-201 at: https://dagshub.com/dev618/End-To-End-Data-Science-with-MLFlow.mlflow/#/experiments/0/runs/bcb0dddf7c4640208d823d02ba2a4ccb
🧪 View experiment at: https://dagshub.com/dev618/End-To-End-Data-Science-with-MLFlow.mlflow/#/experiments/0
